In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas_profiling import ProfileReport
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold, StratifiedKFold, GroupKFold
from tqdm.notebook import tqdm
import datetime
from sklearn.metrics import mean_squared_error, mean_absolute_error
import gc
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
train = pd.read_csv("/kaggle/input/tabular-playground-series-jan-2021/train.csv")
test = pd.read_csv("/kaggle/input/tabular-playground-series-jan-2021/test.csv")
sample_submission = pd.read_csv('../input/tabular-playground-series-jan-2021/sample_submission.csv')

In [ ]:
train.head()

In [ ]:
train.describe()

In [ ]:
train.isna().sum()

In [ ]:
#profile = ProfileReport(train, title='Train Profiling Report', html={'style':{'full_width':True}})

In [ ]:
#profile

In [ ]:
test.head()

In [ ]:
test.describe()

In [ ]:
test.isna().sum()

In [ ]:
sample_submission.head()

In [ ]:
target = train['target'].values

In [ ]:
columns = test.columns[1:]

In [ ]:
np.unique(target).shape


In [ ]:
target.shape

In [ ]:
train_oof = np.zeros((300000,))
test_preds = 0
train_oof.shape

In [ ]:
NUM_FOLDS = 10
kf = KFold(n_splits=NUM_FOLDS, shuffle=True, random_state=0)

for f, (train_ind, val_ind) in tqdm(enumerate(kf.split(train, target))):
        #print(f'Fold {f}')
        train_df, val_df = train.iloc[train_ind][columns], train.iloc[val_ind][columns]
        train_target, val_target = target[train_ind], target[val_ind]
        
        model = MLPRegressor(validation_fraction=0.0, max_iter=110)
        model.fit(train_df, train_target)
        temp_oof = model.predict(val_df)
        temp_test = model.predict(test[columns])

        train_oof[val_ind] = temp_oof
        test_preds += temp_test/NUM_FOLDS

In [ ]:
mean_squared_error(train_oof, target, squared=False)

In [ ]:
np.save('train_oof', train_oof)
np.save('test_preds', test_preds)

In [ ]:
test_preds

In [ ]:
sample_submission['target'] = test_preds
sample_submission.to_csv('submission.csv', index=False)